In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import PyPDF2
import tabula
import os

In [2]:
# URL страницы, с которой будем извлекать ссылки
URL = 'https://www.cbr.ru/statistics/bbs/'
TABLE_PATTERN = re.compile(r".*нефинансовым\s+организациям\s+в\s+рублях")
YEAR_PATTERN = re.compile(r'202\d год')

In [3]:
#Функции для поиска и загрузки последних банковских бюллетеней с сайта ЦБ РФ
def find_specific_links(url):
    # Отправляем GET-запрос на указанный URL
    response = requests.get(url)
    response.raise_for_status()  # Проверяем, что запрос прошел успешно

    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Собираем все ссылки на странице
    links = soup.find_all('a', href=True)

    # Подготавливаем регулярное выражение для поиска нужного формата ссылок
    pattern = re.compile(r'^/Collection/Collection/File/\d+/Bbs\d{4}r\.pdf$')

    # Фильтруем ссылки по регулярному выражению и формируем полные URL
    filtered_links = ['https://www.cbr.ru' + link['href'] for link in links if pattern.match(link['href'])]

    return filtered_links

def download_files(links, num_files=4):
    # Скачиваем только первые `num_files` файлов
    for link in links[:num_files]:
        file_name = link.split('/')[-1]
        save_path = os.path.join('downloaded_files', file_name)

        # Проверяем, существует ли уже файл
        if not os.path.exists(save_path):
            # Отправляем GET-запрос для скачивания файла
            response = requests.get(link)
            response.raise_for_status()

            # Сохраняем файл
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f'Файл {file_name} был успешно скачан и сохранен в {save_path}')
        else:
            print(f'Файл {file_name} уже существует и не будет скачан заново.')

In [4]:
# Создаем папку для сохранения файлов, если она не существует
if not os.path.exists('downloaded_files'):
    os.makedirs('downloaded_files')

# Вызываем функцию для поиска ссылок
specific_links = find_specific_links(URL)

# Вызываем функцию для скачивания файлов
download_files(specific_links)

Файл Bbs2405r.pdf уже существует и не будет скачан заново.
Файл Bbs2404r.pdf уже существует и не будет скачан заново.
Файл Bbs2403r.pdf уже существует и не будет скачан заново.
Файл Bbs2402r.pdf уже существует и не будет скачан заново.


In [5]:
# Путь к PDF файлу
file_path = 'downloaded_files/Bbs2405r.pdf'

# Открываем PDF файл
with open(file_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    found_page = None
    
    # Перебираем страницы от 89 до 106
    for page_number in range(88, 105):
        page = reader.pages[page_number]
        text = page.extract_text()
        
        # Проверяем наличие фразы на странице
        if TABLE_PATTERN.search(text):
            print(f"Фраза 'нефинансовым организациям в рублях' найдена на странице {page_number + 1}")
            found_page = page_number + 1  # Сохраняем номер страницы для дальнейшей обработки
            break
    else:
        print("Фраза 'нефинансовым организациям в рублях' не найдена на указанных страницах.")

# Если фраза найдена, пытаемся извлечь таблицу с этой страницы
if found_page:
    # Используем tabula для извлечения таблиц с найденной страницы
    tables = tabula.read_pdf(file_path, pages=found_page, multiple_tables=True)
    
    # Проверяем, что таблицы найдены
    if tables:
        table = tables[0]  # Предполагаем, что нужная таблица - первая
        df = pd.DataFrame(table)

        # Объединяем строки 0-3
        combined_header = df.iloc[0:4].fillna('').apply(lambda x: ' '.join(x), axis=0)
        
        # Обновляем DataFrame с новым заголовком
        df.columns = combined_header
        df = df.drop([0, 1, 2, 3]).reset_index(drop=True)
        
        # Оставляем только первые 10 столбцов
        df = df.iloc[:, :9]
        
        # Выводим итоговую таблицу
        print(f"Таблица со страницы {found_page}:")
        display(df)
        df.to_csv('merged_table.csv', index=False)  # Сохранение таблицы в файл

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Фраза 'нефинансовым организациям в рублях' найдена на странице 100
Таблица со страницы 100:


,,"до 30 дней, включая “до востре­ бования”",от 31 до 90 дней,от 91 до 180 дней,от 181 дня до 1 года,"до 1 года, включая “до востре­ бования”",от 1 года до 3 лет,свыше 3 лет,свыше 1 года
0,1,2,3,4,5,6,7,8,9
1,2023 год,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,январь,"9,53","9,99","10,85","9,03","9,79","9,22","7,42","8,23"
3,февраль,"9,19","10,15","10,85","8,59","9,54","9,59","7,64","8,95"
4,март,"9,27","10,32","10,70","8,68","9,59","9,55","8,08","8,88"
5,апрель,"9,42","10,20","10,62","9,37","9,86","9,38","8,88","9,11"
6,май,"9,33","10,10","10,88","9,18","9,83","9,75","8,60","9,09"
7,июнь,"9,16","10,10","10,40","8,34","9,46","9,70","9,30","9,47"
8,июль,"9,36","10,63","10,70","8,55","9,71","9,94","8,98","9,37"
9,август,"10,80","11,72","12,72","12,79","12,24","11,34","11,15","11,24"


In [6]:
# Инициализация текущего года
current_year = None

# Проходим по значениям столбца 0 и проверяем наличие обозначения года, обновляем даты соответственно
for i in range(len(df)):
    cell_value = df.iloc[i, 0]
    if YEAR_PATTERN.match(cell_value):
        current_year = YEAR_PATTERN.findall(cell_value)[0][:4]  # Извлекаем год
    else:
        month_mapping = {
            'январь': '01',
            'февраль': '02',
            'март': '03',
            'апрель': '04',
            'май': '05',
            'июнь': '06',
            'июль': '07',
            'август': '08',
            'сентябрь': '09',
            'октябрь': '10',
            'ноябрь': '11',
            'декабрь': '12'
        }
        month_name = cell_value.strip().lower()
        if month_name in month_mapping:
            df.iloc[i, 0] = f"{current_year}-{month_mapping[month_name]}-01"

# Устанавливаем обновленный столбец даты в качестве индекса
df.set_index(df.columns[0], inplace=True)
df = df.dropna()

# Отображаем обновленный DataFrame
display(df)

,"до 30 дней, включая “до востре­ бования”",от 31 до 90 дней,от 91 до 180 дней,от 181 дня до 1 года,"до 1 года, включая “до востре­ бования”",от 1 года до 3 лет,свыше 3 лет,свыше 1 года
,,,,,,,,
1,2,3,4,5,6,7,8,9
2023-01-01,"9,53","9,99","10,85","9,03","9,79","9,22","7,42","8,23"
2023-02-01,"9,19","10,15","10,85","8,59","9,54","9,59","7,64","8,95"
2023-03-01,"9,27","10,32","10,70","8,68","9,59","9,55","8,08","8,88"
2023-04-01,"9,42","10,20","10,62","9,37","9,86","9,38","8,88","9,11"
2023-05-01,"9,33","10,10","10,88","9,18","9,83","9,75","8,60","9,09"
2023-06-01,"9,16","10,10","10,40","8,34","9,46","9,70","9,30","9,47"
2023-07-01,"9,36","10,63","10,70","8,55","9,71","9,94","8,98","9,37"
2023-08-01,"10,80","11,72","12,72","12,79","12,24","11,34","11,15","11,24"
